In [1]:
import itertools
import numpy as np
import torch
from grnewt import NewtonSummary, compute_Hg, nesterov_lrs

In [2]:
"""
P = 10
num_expes = 1000

dct_errors = {'torch eigh': 0.,
              'torch eig': 0.,
              'numpy eigh': 0.,
              'numpy eig': 0.,
              'torch eigh (float64)': 0.}

val = -1e-6
for i in range(num_expes):
    Hm = torch.randn(P, P)
    Hp = Hm.t() @ Hm
    Hd, Hu = torch.linalg.eigh(Hp)
    Hd[0] = val
    H = Hu @ Hd.diag() @ Hu.t()

    dct_errors['torch eigh'] += abs(torch.linalg.eigh(H).eigenvalues[0].item() - val) / num_expes
    dct_errors['torch eig'] += abs(torch.linalg.eig(H).eigenvalues.real.sort().values[0].item() - val) / num_expes
    dct_errors['numpy eigh'] += abs(np.linalg.eigh(H.numpy()).eigenvalues[0] - val) / num_expes
    dct_errors['numpy eig'] += abs(sorted(np.linalg.eig(H.numpy()).eigenvalues.real)[0] - val) / num_expes
    dct_errors['torch eigh (float64)'] += abs(torch.linalg.eigh(H.to(dtype = torch.float64)).eigenvalues[0].item() - val) / num_expes

for k, v in dct_errors.items():
    print(k, ':', v)
"""

"\nP = 10\nnum_expes = 1000\n\ndct_errors = {'torch eigh': 0.,\n              'torch eig': 0.,\n              'numpy eigh': 0.,\n              'numpy eig': 0.,\n              'torch eigh (float64)': 0.}\n\nval = -1e-6\nfor i in range(num_expes):\n    Hm = torch.randn(P, P)\n    Hp = Hm.t() @ Hm\n    Hd, Hu = torch.linalg.eigh(Hp)\n    Hd[0] = val\n    H = Hu @ Hd.diag() @ Hu.t()\n\n    dct_errors['torch eigh'] += abs(torch.linalg.eigh(H).eigenvalues[0].item() - val) / num_expes\n    dct_errors['torch eig'] += abs(torch.linalg.eig(H).eigenvalues.real.sort().values[0].item() - val) / num_expes\n    dct_errors['numpy eigh'] += abs(np.linalg.eigh(H.numpy()).eigenvalues[0] - val) / num_expes\n    dct_errors['numpy eig'] += abs(sorted(np.linalg.eig(H.numpy()).eigenvalues.real)[0] - val) / num_expes\n    dct_errors['torch eigh (float64)'] += abs(torch.linalg.eigh(H.to(dtype = torch.float64)).eigenvalues[0].item() - val) / num_expes\n\nfor k, v in dct_errors.items():\n    print(k, ':', v)\n"

In [3]:
S = 10

g = torch.randn(S)

Hm = torch.randn(S, S)
Hp = Hm.t() @ Hm
Hd, Hu = torch.linalg.eigh(Hp)
Hd[0] = -.000001
H = Hu @ Hd.diag() @ Hu.t()
print('torch eigh = ', torch.linalg.eigh(H).eigenvalues[0].item())
print('torch eig =', torch.linalg.eig(H).eigenvalues.real.sort().values[0].item())
print('numpy eigh =', np.linalg.eigh(H.numpy()).eigenvalues[0])
print('numpy eig =', sorted(np.linalg.eig(H.numpy()).eigenvalues.real)[0])
print('torch eigh (float64) =', torch.linalg.eigh(H.to(dtype = torch.float64)).eigenvalues[0].item())

order3_ = torch.randn(S).abs()
order3_[1] = 0

lrs, logs = nesterov_lrs(H, g, order3_)
print('lrs =', lrs)
print('logs:', logs)

torch eigh =  -3.6789199384656968e-06
torch eig = -1.3899551731810789e-06
numpy eigh = -9.01293e-07
numpy eig = -9.35971e-07
torch eigh (float64) = -9.012929823473555e-07
lrs = tensor([ 0.7345,  0.8709,  1.0126, -2.0833,  0.8786, -0.0838,  0.2544, -0.1824,
        -0.2629, -0.4058])
logs: {'small_eigs': True, 'do_float64': True, 'H_pd': False, 'D_sing': True, 'x0_numerical': 'converged', 'x0': 3.987068402667878e-06, 'f(x0) > 0': True, 'x1': 3.0000119612052085, 'r': 1.0925892287595027, 'r_converged': True, 'found': True, 'time': 0.1318826675415039}


In [4]:
### Building test

def test_nesterov(S, mode_H = 'random', mode_order3_ = 'random', verbose = True):
    g = torch.randn(S)
    
    Hm = torch.randn(S, S)
    Hp = Hm.t() @ Hm
    Hd, Hu = torch.linalg.eigh(Hp)
    if mode_H == 'positive':
        pass
    elif mode_H == 'random':
        Hd.normal_()
    elif mode_H == 'small':
        Hd[0] = -1e-6
    else:
        raise NotImplementedError('')
    H = Hu @ Hd.diag() @ Hu.t()
    
    order3_ = torch.randn(S).abs()
    if mode_order3_ == 'random':
        pass
    elif mode_order3_ == 'zero':
        order3_[1] = 0.
    elif mode_order3_ == 'small':
        order3_[1] = 2e-5
    else:
        raise NotImplementedError('')
    
    lrs, logs = nesterov_lrs(H, g, order3_)
    #print('lrs =', lrs)
    if verbose:
        print('found: {}'.format(logs['found']))
        if logs['found']:
            print('    lrs =', lrs)
        if True:
            print('    logs =', logs)

    return lrs, logs

In [5]:
for P in [2, 5, 10, 20, 30, 50]:
    print('###################')
    print('##### P = {} #####'.format(P))
    print('###################')
    for i in range(5):
        #print('beginning')
        test_nesterov(P, mode_H = 'random', mode_order3_ = 'zero')
        #print('end')
    print('')

###################
##### P = 2 #####
###################
found: False
    logs = {'small_eigs': True, 'do_float64': True, 'H_pd': False, 'D_sing': True, 'x0_numerical': 'not converged', 'found': False, 'time': 0.027151823043823242}
found: False
    logs = {'small_eigs': True, 'do_float64': True, 'H_pd': False, 'D_sing': True, 'x0_numerical': 'not converged', 'found': False, 'time': 0.033498525619506836}
found: True
    lrs = tensor([ 0.2768, -1.4546])
    logs = {'small_eigs': True, 'do_float64': True, 'H_pd': True, 'D_sing': True, 'x0': 0, 'f(x0) > 0': True, 'x1': 1.0, 'r': 0.23670992941673435, 'r_converged': True, 'found': True, 'time': 0.0014605522155761719}
found: True
    lrs = tensor([ 9.9558, 56.9457])
    logs = {'small_eigs': True, 'do_float64': True, 'H_pd': False, 'D_sing': True, 'x0_numerical': 'converged', 'x0': 7.597341458706787, 'f(x0) > 0': True, 'x1': 25.79202437612036, 'r': 8.62462193577644, 'r_converged': True, 'found': True, 'time': 0.003261566162109375}
found: Fal

In [7]:
num_expes = 200

for P in [2, 5, 10, 20, 30, 50]:
    print('###################')
    print('##### P = {} #####'.format(P))
    print('###################')
    for mode_H, mode_order3_ in itertools.product(['positive', 'random', 'small'], ['random', 'small', 'zero']):
        found = 0
        tot_time = 0.
        for i in range(num_expes):
            #print('beginning')
            lrs, logs = test_nesterov(P, mode_H = mode_H, mode_order3_ = mode_order3_, verbose = False)
            tot_time += logs['time']
            if logs['found']:
                found += 1
            #print('end')
        print('mode_H = {}, mode_order3_ = {}: {:.2%} success, {:.6f} seconds'.format(mode_H, mode_order3_, 
                                                                                      found / num_expes, tot_time))
    print('')

###################
##### P = 2 #####
###################
mode_H = positive, mode_order3_ = random: 100.00% success, 0.489216 seconds
mode_H = positive, mode_order3_ = small: 100.00% success, 0.450472 seconds
mode_H = positive, mode_order3_ = zero: 100.00% success, 0.265254 seconds
mode_H = random, mode_order3_ = random: 100.00% success, 0.509266 seconds
mode_H = random, mode_order3_ = small: 98.00% success, 0.419339 seconds
mode_H = random, mode_order3_ = zero: 47.50% success, 2.225386 seconds
mode_H = small, mode_order3_ = random: 100.00% success, 0.254981 seconds
mode_H = small, mode_order3_ = small: 100.00% success, 0.263143 seconds
mode_H = small, mode_order3_ = zero: 100.00% success, 0.275708 seconds

###################
##### P = 5 #####
###################
mode_H = positive, mode_order3_ = random: 100.00% success, 0.322450 seconds
mode_H = positive, mode_order3_ = small: 100.00% success, 0.335235 seconds
mode_H = positive, mode_order3_ = zero: 100.00% success, 0.204485 seconds
